In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist =input_data.read_data_sets("./MNIST/data/" ,one_hot = True)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST/data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
X = tf.placeholder(tf.float32, [None,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])

keep_prob = tf.placeholder(tf.float32)

#convolution 계층
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)

L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W3 = tf.Variable(tf.random_normal([7*7*64,256],stddev=0.01))
L3 = tf.reshape(L2,[-1,7*7*64])
L3 = tf.matmul(L3,W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256,10],stddev = 0.01))
model = tf.matmul(L3,W4)

cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model,labels=Y))

optimizer = tf.train.AdamOptimizer(0.001).mnimize(cost)

init =tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        batch_xs = batch_xs.reshape(-1,28,28,1)
        _,cost_val = sess.run([optimizer,cost], feed_dict = {X: batch_xs, Y: batch_ys#, keep_prob = 0.8})
        total_cost = total_cost + cost_val
                                                             
    print("Epoch",'%0.4d'%(epoch+1),'Avg.cost=','{:.3f}'.format(total_cost/ total_batch))
print("최적화 완료")

is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy = tf. reduce_mean(tf.cast(is_correct,tf.float32))
print('정확도:', sess.run(accuracy,feed_dict={X:mnist.test.images.reshape(-1,28,28,1),
                                          Y: mnist.test.labels, keep_prob :1}))

SyntaxError: invalid syntax (<ipython-input-4-5d6e7aa831ce>, line 46)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
learning_rate = 0.01
training_epoch = 20
batch_size = 100
n_hidden = 256
n_input = 28*28

#encoder

X = tf.placeholder(tf.float32,[None,n_input])
W_encode = tf.Variable(tf.random_normal([n_input,n_hidden]))
b_encode = tf.Variable(tf.random_normal([n_hidden]))

encoder = tf.nn.sigmoid(tf.add(tf.matmul(X,W_encode),b_encode))


#decoder

W_decode = tf.Variable(tf.random_normal([n_hidden,n_input]))
b_decode = tf.Variable(tf.random_normal([n_input]))

decoder = tf.nn.sigmoid(tf.add(tf.matmul(encoder, W_decode), b_decode))

cost = tf.reduce_mean((tf.pow(X-decoder,2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

totabl_batch = int(mnist.train.num_examples/ batch_size)

for epoch in range(training_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer,cost],feed_dict={X: batch_xs})
        total_cost += cost_val
        
        
    print('epoch:','%0.4d'%(epoch +1), 'Avg.cost=','{%4f}'.format(total_cost/total_batch))
    
print('최적화 완료!')



In [ ]:
sample_size = 10
samples = sess.run(decoder, feed_dict = {X:mnist.test.images[:sample_size]})
fig,ax = plt.subplot(2, sample_size, figsize = (sample_size,2))

for i in range(sample_size):
    ax[0][i].set_axis_off()
    ax[1][i].set_axis_off()
    ax[1][i].imshow(np.reshape(mnist.test.images[i],(28,28)))
    ax[1][i].imshow(np.reshape(samples[i],(28,28)))


plt.show()